In [ ]:
import clip
import ruclip
import torch
import torchvision.transforms as transforms

import json
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from matplotlib import pyplot as plt

import utils

random_seed = 17
torch.manual_seed(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

# Получение эмбеддингов изображений

Запустим каждую модель для получения эмбеддингов изображений и текстов из тестовой выборки. Получим таким образом для каждой модели базу вычисленных эмбеддингов.

In [2]:
# определение доступного устройства
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Current device:', device)

# чтение данных тестовой выборки
df = pd.read_csv('data/test_data.csv')
print('Test data size:', df.shape)
df.head()

Current device: cuda
Test data size: (328, 7)


,Unnamed: 0,local_image_path,name,category,text,url,id
0,12,data/embroteka_imgs/12.png,астронавт,астрология космос,астронавт астрология космос,https://embroteka.ru/astronavt-16946,12
1,21,data/embroteka_imgs/21.jpg,гагарин ю.а.,астрология космос люди,гагарин ю.а. астрология космос люди,https://embroteka.ru/gagarin,21
2,32,data/embroteka_imgs/32.png,звезды,астрология космос,звезды астрология космос,https://embroteka.ru/dve-zvezdi,32
3,56,data/embroteka_imgs/57.jpg,знак зодиака козерог,астрология космос,знак зодиака козерог астрология космос,https://embroteka.ru/capricorn,56
4,57,data/embroteka_imgs/58.jpg,знак зодиака козерог,астрология космос,знак зодиака козерог астрология космос,https://embroteka.ru/zodiak5,57


## CLIP

In [4]:
# model: CLIP
# preprocess: Resize(224), CenterCrop(), ToTensor(), Normalize()
model_name = 'clip'
model, preprocess = clip.load('ViT-B/32', device=device)
total_image_embeddings, total_text_embeddings = utils.get_full_embeddings(df, model, preprocess, model_name, device)

100%|██████████| 3/3 [00:04<00:00,  1.52s/it]


In [5]:
# формируем структуру данных
data = {
    'ids': df['id'].to_list(), 
    'image_paths': df['local_image_path'].to_list(),
    'image_embeddings': total_image_embeddings,
    'texts': df['text'].to_list(),
    'text_embeddings': total_text_embeddings,
    'urls': df['url'].to_list()
}

# сохраняем в пикл
with open('embeddings/test_embeddings_clip_pretrained.pkl', 'wb') as file:
    pickle.dump(data, file)

## ruCLIP

In [6]:
# model: CLIP
# processor: RuCLIPProcessor для изображений и текста, после вызова возвращает dict
model_name = 'ruclip'
model, processor = ruclip.load('ruclip-vit-base-patch32-224', device=device)
total_image_embeddings, total_text_embeddings = utils.get_full_embeddings(df, model, processor, model_name, device)

/home/student/kholicheva/.venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)
/home/student/kholicheva/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:672: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
100%|██████████| 3/3 [00:03<00:00,  1.12s/it]


In [7]:
# формируем структуру данных
data = {
    'ids': df['id'].to_list(),
    'image_paths': df['local_image_path'].to_list(),
    'image_embeddings': total_image_embeddings,
    'texts': df['text'].to_list(),
    'text_embeddings': total_text_embeddings,
    'urls': df['url'].to_list()
}

# сохраняем в пикл
with open('embeddings/test_embeddings_ruclip_pretrained.pkl', 'wb') as file:
    pickle.dump(data, file)

# Поиск картинок по текстовому запросу

In [3]:
# загружаем json-файл с тестовыми запросами и релевантными изображениями
with open('semantic_test_queries.json') as file:
    queries_and_relevants = json.load(file)
    queries = list(queries_and_relevants.keys())

# приведем к более удобной структуре
for query in queries:
    df_query = pd.DataFrame(queries_and_relevants[query])
    ids = df_query['id'].tolist()
    paths = df_query['local_image_path'].tolist()
    texts = df_query['text'].tolist()
    queries_and_relevants[query] = {'ids': ids, 'paths': paths, 'texts': texts}

## CLIP embeddings

In [9]:
# загружаем эмбеддинги из предобученной модели
with open('embeddings/test_embeddings_clip_pretrained.pkl', 'rb') as file:
    test_data_clip = pickle.load(file)

# инициализируем предобученную модель
model_name = 'clip'
model, preprocess = clip.load('ViT-B/32', device=device)

# посчитаем среднее косинусное сходство эмбеддингов в правильных парах
utils.mean_similarity_between_true(model, test_data_clip['image_embeddings'], test_data_clip['text_embeddings'])

Среднее косинусное сходство эмбеддингов у правильных пар: 0.0315


0.031494140625

In [10]:
# проходим по всем тестовым запросам
queries_and_preds = {}
for query in queries:
    text_embedding = utils.get_text_embedding(query, model, preprocess, model_name, device)
    scores = torch.cosine_similarity(text_embedding, test_data_clip['image_embeddings'], dim=-1)
    top_images_indices = torch.topk(scores, k=5).indices
    ids = np.array(test_data_clip['ids'])[top_images_indices].tolist()
    paths = np.array(test_data_clip['image_paths'])[top_images_indices].tolist()
    texts = np.array(test_data_clip['texts'])[top_images_indices].tolist()
    queries_and_preds[query] = {'ids': ids, 'paths': paths, 'texts': texts}

In [11]:
utils.precision_at_k(queries_and_relevants, queries_and_preds)

----- Precision@5 -----
Запрос "котенок":  0.40
Запрос "петух курица":  0.00
Запрос "девушка":  0.00
Запрос "новый год":  0.20
Запрос "пасха":  0.00
Запрос "цветы ромашки":  0.00
Запрос "иероглифы":  0.00
Запрос "персонажи мультфильмов":  0.00
Запрос "космос":  0.20
Запрос "необычные птицы":  0.00
Запрос "ученый и наука":  0.00
Запрос "надписи буквами":  0.00
Запрос "военная":  1.00
Запрос "автомобили машины":  0.00
Запрос "гарри поттер":  0.00
Запрос "бабочка":  0.00
Запрос "собака играет":  0.00
Запрос "знаки зодиака":  0.00
Запрос "лило и стич":  0.00
Запрос "самолеты небо":  0.00
Запрос "детские рисунки":  0.00
Запрос "любовь":  0.00
Запрос "динозавр":  0.00
Запрос "пиво":  0.20
Среднее значение по всем запросам: 0.08333333333333333


In [12]:
utils.recall_at_k(queries_and_relevants, queries_and_preds)

----- Recall@5 -----
Запрос "котенок":  0.12
Запрос "петух курица":  0.00
Запрос "девушка":  0.00
Запрос "новый год":  0.07
Запрос "пасха":  0.00
Запрос "цветы ромашки":  0.00
Запрос "иероглифы":  0.00
Запрос "персонажи мультфильмов":  0.00
Запрос "космос":  0.06
Запрос "необычные птицы":  0.00
Запрос "ученый и наука":  0.00
Запрос "надписи буквами":  0.00
Запрос "военная":  0.33
Запрос "автомобили машины":  0.00
Запрос "гарри поттер":  0.00
Запрос "бабочка":  0.00
Запрос "собака играет":  0.00
Запрос "знаки зодиака":  0.00
Запрос "лило и стич":  0.00
Запрос "самолеты небо":  0.00
Запрос "детские рисунки":  0.00
Запрос "любовь":  0.00
Запрос "динозавр":  0.00
Запрос "пиво":  0.09
Среднее значение по всем запросам: 0.0281761063011063


## ruCLIP embeddings

In [13]:
# загружаем данные
with open('embeddings/test_embeddings_ruclip_pretrained.pkl', 'rb') as file:
    test_data_ruclip = pickle.load(file)

# инициализируем модель
model_name = 'ruclip'
model, preprocess = ruclip.load('ruclip-vit-base-patch32-224', device=device)

# посчитаем среднее косинусное сходство эмбеддингов в правильных парах
utils.mean_similarity_between_true(model, test_data_ruclip['image_embeddings'], test_data_ruclip['text_embeddings'])

Среднее косинусное сходство эмбеддингов у правильных пар: 0.1874


0.1873556524515152

In [14]:
# проходим по всем тестовым запросам
queries_and_preds = {}
for query in queries:
    text_embedding = utils.get_text_embedding(query, model, preprocess, model_name, device)
    scores = torch.cosine_similarity(text_embedding, test_data_ruclip['image_embeddings'], dim=-1)
    top_images_indices = torch.topk(scores, k=5).indices
    ids = np.array(test_data_ruclip['ids'])[top_images_indices].tolist()
    paths = np.array(test_data_ruclip['image_paths'])[top_images_indices].tolist()
    texts = np.array(test_data_ruclip['texts'])[top_images_indices].tolist()
    queries_and_preds[query] = {'ids': ids, 'paths': paths, 'texts': texts}

In [15]:
utils.precision_at_k(queries_and_relevants, queries_and_preds)

----- Precision@5 -----
Запрос "котенок":  1.00
Запрос "петух курица":  1.00
Запрос "девушка":  0.60
Запрос "новый год":  1.00
Запрос "пасха":  1.00
Запрос "цветы ромашки":  1.00
Запрос "иероглифы":  0.80
Запрос "персонажи мультфильмов":  0.60
Запрос "космос":  1.00
Запрос "необычные птицы":  0.80
Запрос "ученый и наука":  0.20
Запрос "надписи буквами":  0.40
Запрос "военная":  0.60
Запрос "автомобили машины":  1.00
Запрос "гарри поттер":  1.00
Запрос "бабочка":  1.00
Запрос "собака играет":  0.80
Запрос "знаки зодиака":  0.80
Запрос "лило и стич":  1.00
Запрос "самолеты небо":  1.00
Запрос "детские рисунки":  0.20
Запрос "любовь":  0.80
Запрос "динозавр":  1.00
Запрос "пиво":  1.00
Среднее значение по всем запросам: 0.8166666666666668


In [16]:
utils.recall_at_k(queries_and_relevants, queries_and_preds)

----- Recall@5 -----
Запрос "котенок":  0.31
Запрос "петух курица":  0.31
Запрос "девушка":  0.21
Запрос "новый год":  0.36
Запрос "пасха":  0.33
Запрос "цветы ромашки":  0.33
Запрос "иероглифы":  0.57
Запрос "персонажи мультфильмов":  0.19
Запрос "космос":  0.28
Запрос "необычные птицы":  0.29
Запрос "ученый и наука":  0.11
Запрос "надписи буквами":  0.12
Запрос "военная":  0.20
Запрос "автомобили машины":  0.45
Запрос "гарри поттер":  0.83
Запрос "бабочка":  0.33
Запрос "собака играет":  0.25
Запрос "знаки зодиака":  0.25
Запрос "лило и стич":  0.33
Запрос "самолеты небо":  0.33
Запрос "детские рисунки":  0.07
Запрос "любовь":  0.22
Запрос "динозавр":  0.42
Запрос "пиво":  0.45
Среднее значение по всем запросам: 0.3154002825877826
